In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multi-class-object-detection-challenge/convert_preds_to_csv.py
/kaggle/input/multi-class-object-detection-challenge/Kaggle_3LC.ipynb
/kaggle/input/multi-class-object-detection-challenge/yolo_params.yaml
/kaggle/input/multi-class-object-detection-challenge/predict.py
/kaggle/input/multi-class-object-detection-challenge/Starter_Dataset/val/labels/IMG_9300.txt
/kaggle/input/multi-class-object-detection-challenge/Starter_Dataset/val/labels/IMG_9960.txt
/kaggle/input/multi-class-object-detection-challenge/Starter_Dataset/val/labels/IMG_8710.txt
/kaggle/input/multi-class-object-detection-challenge/Starter_Dataset/val/labels/000000010.txt
/kaggle/input/multi-class-object-detection-challenge/Starter_Dataset/val/labels/IMG_9895.txt
/kaggle/input/multi-class-object-detection-challenge/Starter_Dataset/val/labels/IMG_9270.txt
/kaggle/input/multi-class-object-detection-challenge/Starter_Dataset/val/labels/000000007.txt
/kaggle/input/multi-class-object-detection-challenge/Starter_Datas

In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
import yaml
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm  


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
data_config = {
    'path': '/kaggle/input/multi-class-object-detection-challenge/Starter_Dataset',
    'train': 'train/images',  
    'val': 'val/images',      
    'nc': 2,
    'names': ['cheerios', 'soup']
}

with open('data.yaml', 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

In [7]:
print(" FINAL TRAINING: Using Optimized Parameters")
print("Resolution: 960px (best performance)")
print("Augmentation: Color-focused strategy (best for synthetic-to-real)")
print("Training: Extended epochs for maximum performance")

 FINAL TRAINING: Using Optimized Parameters
Resolution: 960px (best performance)
Augmentation: Color-focused strategy (best for synthetic-to-real)
Training: Extended epochs for maximum performance


In [8]:
model = YOLO("yolov8m.pt")

In [9]:
final_results = model.train(
    data='data.yaml',
    
    # Training configuration
    epochs=50,              # xxtended from previous experiments (vs 25 baseline, 30 experiments)
    batch=8,                # optimized for 960px resolution
    imgsz=960,              # optimal resolution 
    device=0,
    patience=50,            # patient training for maximum performance
    
    # Training optimization
    optimizer='SGD',        # proven for YOLO
    lr0=0.001,             # stable learning rate
    cos_lr=True,           # smooth learning rate schedule
    
    # best color-focused augmentation strategy
    hsv_h=0.05, hsv_s=0.7, hsv_v=0.5,  
    fliplr=0.5, flipud=0.1,              
    translate=0.1, scale=0.4, shear=0.02, 
    mixup=0.15, copy_paste=0.1,          
    
    # Project organization
    project='final_optimized_model',
    name='final_run',
    save=True,
    plots=True,
    verbose=True
)


Ultralytics 8.3.175 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.05, hsv_s=0.7, hsv_v=0.5, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.15, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=final_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False,

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 100.0±54.1 MB/s, size: 5119.2 KB)


train: Scanning /kaggle/input/multi-class-object-detection-challenge/Starter_Dataset/train/labels... 1000 images, 57 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:15<00:00, 64.08it/s]


WARNING ⚠️ train: Cache directory /kaggle/input/multi-class-object-detection-challenge/Starter_Dataset/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.2±0.3 ms, read: 137.0±15.2 MB/s, size: 20471.7 KB)


...

      50/50      7.04G     0.1791      0.148     0.7871          9        960: 100%|██████████| 125/125 [01:16<00:00,  1.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.72it/s]

                   all         95        172      0.889       0.86      0.918      0.847



50 epochs completed in 1.129 hours.
Optimizer stripped from final_optimized_model/final_run/weights/last.pt, 52.1MB
Optimizer stripped from final_optimized_model/final_run/weights/best.pt, 52.1MB

Validating final_optimized_model/final_run/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:12<00:00,  2.02s/it]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         95        172      0.919      0.865      0.928      0.853
              cheerios         88         88      0.963      0.894       0.95      0.887
                  soup         84         84      0.875      0.835      0.907      0.819
Speed: 0.4ms preprocess, 19.4ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to final_optimized_model/final_run


In [10]:
final_map50 = final_results.results_dict['metrics/mAP50(B)']
final_map50_95 = final_results.results_dict['metrics/mAP50-95(B)']
precision = final_results.results_dict['metrics/precision(B)']
recall = final_results.results_dict['metrics/recall(B)']

print(f" FINAL PERFORMANCE:")
print(f"   mAP@50: {final_map50:.4f} ({final_map50:.1%})")
print(f"   mAP@50-95: {final_map50_95:.4f}")
print(f"   Precision: {precision:.4f}")
print(f"   Recall: {recall:.4f}")


 FINAL PERFORMANCE:
   mAP@50: 0.9284 (92.8%)
   mAP@50-95: 0.8533
   Precision: 0.9193
   Recall: 0.8648


In [15]:
baseline_performance = 0.7178
improvement = ((final_map50 - baseline_performance) / baseline_performance) * 100
print(f" total Improvement vs Baseline: {improvement:+.1f}%")


 total Improvement vs Baseline: +29.3%


In [16]:
test_dir = '/kaggle/input/multi-class-object-detection-challenge/testImages/images'

In [21]:
len(os.listdir('/kaggle/input/multi-class-object-detection-challenge/testImages/images'))

229

In [27]:
allowed_extensions = ('.jpg', '.png', '.jpeg', '.JPG', '.PNG', '.JPEG')
test_images = []
for ext in allowed_extensions:
    test_images.extend(Path(test_dir).glob(f'*{ext}'))

print(f"Found {len(test_images)} test images in {test_dir}")

Found 229 test images in /kaggle/input/multi-class-object-detection-challenge/testImages/images


In [28]:
results_list = []


In [29]:
test_images

[PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9946.jpg'),
 PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9953.jpg'),
 PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9951.jpg'),
 PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9901.jpg'),
 PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9913.jpg'),
 PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9937.jpg'),
 PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9929.jpg'),
 PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9966.jpg'),
 PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9914.jpg'),
 PosixPath('/kaggle/input/multi-class-object-detection-challenge/testImages/images/IMG_9958.jpg'),
 PosixPath

In [30]:
for img_path in tqdm(test_images, desc="Generating predictions"):
    try:
        # Load image to get dimensions
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"could not load {img_path}")
            continue
            
        img_height, img_width = img.shape[:2]
        
        
        results = model(img_path, conf=0.01, device=0, verbose=False)
        
        
        if results[0].boxes is not None and len(results[0].boxes) > 0:
            prediction_strings = []
            
            for box in results[0].boxes:
                try:
                    # extract class and confidence
                    cls = int(box.cls.cpu().numpy().item())
                    conf = float(box.conf.cpu().numpy().item())
                    
                    # extract coordinates using working pattern
                    box_data = box.xywh[0].cpu().numpy()  # Convert to numpy first
                    
                    # Extract normalized coordinates
                    x_center = box_data[0] / img_width
                    y_center = box_data[1] / img_height
                    w = box_data[2] / img_width
                    h = box_data[3] / img_height
                    
                    # validate coordinates are in [0,1] range
                    if all(0 <= v <= 1 for v in [x_center, y_center, w, h]):
                        # Format: class conf x_center y_center width height
                        prediction_strings.append(f"{cls} {conf:.6f} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")
                    else:
                        print(f"warning: invalid coordinates for {img_path.name}")
                        
                except Exception as box_error:
                    print(f"warning: error processing box in {img_path.name}: {box_error}")
                    continue
            
            prediction_str = " ".join(prediction_strings) if prediction_strings else "no boxes"
        else:
            prediction_str = "no boxes"
        
        results_list.append({
            "image_id": img_path.stem,  
            "prediction_string": prediction_str
        })
        
    except Exception as img_error:
        print(f"Error processing {img_path}: {img_error}")
        results_list.append({
            "image_id": img_path.stem,
            "prediction_string": "no boxes"
        })


Generating predictions: 100%|██████████| 229/229 [03:18<00:00,  1.16it/s]


In [35]:
submission_df = pd.DataFrame(results_list)

# Display submission statistics
print(f"\n Submission Statistics:")
print(f"Total images: {len(submission_df)}")
print(f"Images with predictions: {(submission_df['prediction_string'] != 'no boxes').sum()}")
print(f"Images with no detections: {(submission_df['prediction_string'] == 'no boxes').sum()}")


 Submission Statistics:
Total images: 229
Images with predictions: 228
Images with no detections: 1


In [36]:
print(f"\n Sample Submission Format:")
print(submission_df.head().to_string(index=False))


 Sample Submission Format:
image_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    prediction_string
IMG_9946                                                                                                                                                                                                                                                                                                                                          0 0.735331 0.279929 0.059081 0.219726 0.118162 1 0.104773 0.848774 0.306784 0.302453 0.613567 1 0.078889 0.1

In [37]:
print("\n OPTIMIZATION JOURNEY SUMMARY")
print("="*60)
print("Systematic Parameter Optimization Results:")
print(f"  Baseline (640px, minimal aug):     71.78% mAP@50")
print(f"  + Resolution optimization (960px): 72.78% mAP@50 (+1.39%)")
print(f"  + Color-focused augmentation:      79.32% mAP@50 (+10.50%)")
print(f"  + Extended training (50 epochs):   {final_map50:.2%} mAP@50")
print("="*60)


 OPTIMIZATION JOURNEY SUMMARY
Systematic Parameter Optimization Results:
  Baseline (640px, minimal aug):     71.78% mAP@50
  + Resolution optimization (960px): 72.78% mAP@50 (+1.39%)
  + Color-focused augmentation:      79.32% mAP@50 (+10.50%)
  + Extended training (50 epochs):   92.84% mAP@50


In [1]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


NameError: name 'submission_df' is not defined